In [16]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
#data processing
path = 'data_rct/'
df_train_txt = pd.read_csv(path + 'training_text', sep='\|\|', header=None, skiprows=1, names=["ID","Text"])
df_train_var = pd.read_csv(path + 'training_variants')
df_test_txt = pd.read_csv(path + 'stage2_test_text.csv', sep='\|\|', header=None, skiprows=1, names=["ID","Text"])
df_test_var = pd.read_csv(path + 'stage2_test_variants.csv')
df_train = pd.merge(df_train_var, df_train_txt, how='left', on='ID')
df_test = pd.merge(df_test_var, df_test_txt, how='left', on='ID')
col = ['ID', 'Gene', 'Variation', 'Text', 'Class']
df_train = df_train.loc[:, col]
df_train.head()

/home/quoniammm/anaconda3/envs/py3Tfgpu/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/quoniammm/anaconda3/envs/py3Tfgpu/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


,ID,Gene,Variation,Text,Class
0,0,FAM58A,Truncating Mutations,Cyclin-dependent kinases (CDKs) regulate a var...,1
1,1,CBL,W802*,Abstract Background Non-small cell lung canc...,2
2,2,CBL,Q249E,Abstract Background Non-small cell lung canc...,2
3,3,CBL,N454D,Recent evidence has demonstrated that acquired...,3
4,4,CBL,L399V,Oncogenic mutations in the monomeric Casitas B...,4


In [22]:
df_train['Text'][3]

'Recent evidence has demonstrated that acquired uniparental disomy (aUPD) is a novel mechanism by which pathogenetic mutations in cancer may be reduced to homozygosity. To help identify novel mutations in myeloproliferative neoplasms (MPNs), we performed a genome-wide single nucleotide polymorphism (SNP) screen to identify aUPD in 58 patients with atypical chronic myeloid leukemia (aCML; n = 30), JAK2 mutation–negative myelofibrosis (MF; n = 18), or JAK2 mutation–negative polycythemia vera (PV; n = 10). Stretches of homozygous, copy neutral SNP calls greater than 20Mb were seen in 10 (33%) aCML and 1 (6%) MF, but were absent in PV. In total, 7 different chromosomes were involved with 7q and 11q each affected in 10% of aCML cases. CBL mutations were identified in all 3 cases with 11q aUPD and analysis of 574 additional MPNs revealed a total of 27 CBL variants in 26 patients with aCML, myelofibrosis or chronic myelomonocytic leukemia. Most variants were missense substitutions in the RING

In [4]:
title = df_train['Gene'].values + ' ' + df_train['Variation'].values
title = title.reshape(len(title), 1).tolist()

In [5]:
content = df_train['Text'].values
content = content.reshape(len(content), 1).tolist()

In [6]:
cla = df_train['Class'].values
cla = cla.reshape(len(cla), 1).tolist()

In [7]:
class Sentence:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.n_words = 2 # Count SOS and EOS
      
    def index_words(self, sentence):
        for word in sentence.split(' '):
            self.index_word(word)

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [8]:
tit = Sentence('title')
con = Sentence('content')
for t, c in zip (title, content):
    c = ''.join(c)
#     print(c)
    t = ''.join(t)
#     print(t)
    tit.index_words(t)
    con.index_words(c)

print('tit: {}'.format(tit.n_words))
print('con: {}'.format(con.n_words))

tit: 3281
con: 496037


In [9]:
def indexes_from_sentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def variable_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)
    var = Variable(torch.LongTensor(indexes).view(-1, 1)).cuda()
#     if USE_CUDA: var = var.cuda()
    return var

def variables_from_pair(t, c):
    title = variable_from_sentence(tit, t)
    content = variable_from_sentence(con, c)
    return (title, content)

In [10]:
# USE_CUDA = True

In [11]:
# class LinearText(nn.Module): 
#     def __init__(self):
#         super(CNNText, self).__init__()
#         self.encoder_tit = nn.Embedding(3281, 64)
#         self.encoder_con = nn.Embedding(496037, 512)
        
#         self.title_li = nn.Linear(64, 32) # 1 * len(tit) * 32
#         self.con_li = nn.Linear(512, 256) # 1 * len(con) * 256
# #         t.cat((content_out_3,content_out_4,content_out_5),dim=1)
#         self.weight = nn.Parameters()
# #         self.fc = nn.Linear(input_size, 9)

#     def forward(self, title, content):
#         title = self.encoder_tit(title)
#         content = self.encoder_con(content)
        
#         tit_li = self.title_li(title)
#         con_li = self.con_li(content)
        
#         max_tit_li = F.max_pool2d(tit_li, kernel_size=(1, 32))
#         print(max_tit_li.size())
#         max_con_li = F.max_pool2d(con_li, kernel_size=(1, 256))
#         print(max_con_li.size())
        
#         max_add = torch.cat((max_tit_li, max_con_li), dim=1)
#         print(max_add.size())
        
#         logits = F.linear()
#         return F.log_softmax(logits)

In [12]:
# input = Variable(torch.randn((1,15,20)))
# print(input)
# F.max_pool2d(input, (1,20))

In [13]:
# cnnt = LinearText()

# optimizer = optim.Adam(cnnt.parameters(), lr=.001)
# Loss = nn.NLLLoss()

# for epoch in range(50):
#     loss = 0
    
#     t = ''.join(title[epoch])
#     c = ''.join(content[epoch])
#     T, C = variables_from_pair(t, c)
# #     print(T.squeeze(1).unsqueeze(0))
#     T = T.squeeze(1).unsqueeze(0)
#     C = C.squeeze(1).unsqueeze(0)
#     print(T.size())
#     print(C.size())
#     optimizer.zero_grad()
    
#     out = cnnt(T, C)
#     target = cla[epoch]
#     loss += Loss(out, target)
    
#     loss.backward()
#     optimizer.step()
    
# print("Loss is {} at {} epoch".format(loss, epoch))

In [14]:
class CNNText(nn.Module): 
    def __init__(self):
        super(CNNText, self).__init__()
        self.encoder_tit = nn.Embedding(3281, 64)
        self.encoder_con = nn.Embedding(496037, 256)
        self.test = nn.Parameter(torch.LongTensor(0))
        
        self.title_conv_1 = nn.Sequential(
            nn.Conv1d(in_channels = 64,
                      out_channels = 1,
                      kernel_size = 1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=1),
        )
        
        self.title_conv_2 = nn.Sequential(
            nn.Conv1d(in_channels = 64,
                      out_channels = 1,
                      kernel_size = 2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=1),
        )

        self.content_conv_3 = nn.Sequential(
            nn.Conv1d(in_channels = 256,
                      out_channels = 1,
                      kernel_size = 3),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size = 50)
        )
        
        self.content_conv_4 = nn.Sequential(
            nn.Conv1d(in_channels = 256,
                      out_channels = 1,
                      kernel_size = 4),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size = 50)
        )
            
        self.content_conv_5 = nn.Sequential(
            nn.Conv1d(in_channels = 256,
                      out_channels = 1,
                      kernel_size = 5),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size = 50)
        )
        
        
            
        self.fc = nn.Linear(64, 9)

    def forward(self, title, content):
        title = self.encoder_tit(title)
        title = title.permute(0,2,1)
#         print(title.size())
#         print(title.unsqueeze(1).size())
#         title_rehape = torch.index_select(title, 2, Variable(torch.LongTensor([0,2,1])))
#         print("title_reshape size is {}".format(title_rehape.size()))
        title_out_1 = self.title_conv_1(title)
#         print(title_out_1.size())
        title_out_2 = self.title_conv_2(title)
#         print(title_out_2.size())
        
        content = self.encoder_con(content)
        content = content.permute(0,2,1)
#         self.test = torch.LongTensor(50)
#         content_reshape = torch.index_select(content, 2, Variable(torch.LongTensor([0,2,1])))
#         print(content.size())
        content_out_3 = self.content_conv_3(content)
#         print(content_out_3.size())
        
        content_out_4 = self.content_conv_4(content)
#         print(content_out_4.size())
        
        content_out_5 = self.content_conv_5(content)
#         print(content_out_5.size())
        
            
        conv_out = torch.cat((
            title_out_1,
            title_out_2,
            content_out_3,
            content_out_4,
            content_out_5
        ),dim=2)
        
#         print(conv_out.size())
        conv_out, _ = conv_out.topk(64, dim=2)
#         print(conv_out.size())
#         kernel_size = conv_out.size()[2] - 255
#         print(kernel_size)
#         print(conv_out.size())
#         conv_out = torch.cat((content_out_3,content_out_4,content_out_5),dim=1)
        logits = self.fc(conv_out.squeeze(0))
#         print(logits.size())
#         print(logits)
#         print(F.softmax(logits.squeeze(0)))
#         print(F.log_softmax(logits.squeeze(0)))
        return F.log_softmax(logits.squeeze(0))


In [15]:
cnnt = CNNText()
cnnt.cuda()
optimizer = optim.Adam(cnnt.parameters(), lr=.001)
Loss = nn.NLLLoss()

for epoch in range(50):
    loss = 0
    
    t = ''.join(title[epoch])
    c = ''.join(content[epoch])
    T, C = variables_from_pair(t, c)
    
    T = T.squeeze(1).unsqueeze(0)
    C = C.squeeze(1).unsqueeze(0)
    optimizer.zero_grad()
    
    out = cnnt(T, C)
#     print(out)
#     print(torch.LongTensor(cla[epoch]))
    target = Variable(torch.LongTensor(cla[epoch])).cuda()
    print(out)
    print(target)
    loss += Loss(out, target)
    
    loss.backward()
    optimizer.step()
    
    if not epoch % 10:
        print("Loss is {} at {} epoch".format(loss, epoch))

torch.Size([1, 1, 256])
Variable containing:
-2.8413 -1.4480 -1.3816 -3.7719 -2.1339 -2.2044 -2.8442 -2.3916 -2.9177
[torch.cuda.FloatTensor of size 1x9 (GPU 0)]

Variable containing:
 1
[torch.cuda.LongTensor of size 1 (GPU 0)]



RuntimeError: cuda runtime error (2) : out of memory at /pytorch/torch/lib/THC/generic/THCStorage.cu:66